In this notebook we will:
- Use Albumentations to create a new augmented datasets with thousands of augmented images
- Fine tune the EfficientNetV2 model for maximum F1 score and the final deployment of the model to our website

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Hyperparameters
IMAGE_SIZE = 384
BATCH_SIZE = 8
CHANNELS= 3
ES_EPOCHS = 20 
CLASSES = 10
EPOCHS= 200
SAMPLES_AUG_IMG = 2

In [ ]:
CLASS_NAMES = ["1989", "Acoustic", "Fearless", "Folkmore", "Lover", "Midnights", "Red", "Reputation", "Speak Now", "TTPD"]

In [ ]:
# Train
train_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/train"

# Val
val_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/val"

# Test
test_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/test"

Data augmentation pipeline using Albumentations 

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

In [ ]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
image = cv2.imread('Images_App_Test/1989_orange.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
visualize(image)

In [ ]:
transforms = A.Compose([
            A.Rotate(limit=40),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.HorizontalFlip(),
            A.ShiftScaleRotate(p=0.5),
            A.Blur(blur_limit=3),
            A.OpticalDistortion(),
            A.GridDistortion(),
        ])

In [ ]:
import os
import cv2
import random
import albumentations as A

# Define paths
dataset_path = 'train'  # Replace with your dataset path
augmented_dataset_path = 'augmented_train'  # Replace with desired output path

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),              # Horizontal flip with 50% probability
    A.VerticalFlip(p=0.5),                # Vertical flip with 50% probability
    A.RandomRotate90(p=0.5),              # Random 90-degree rotation with 50% probability
    A.RandomBrightnessContrast(p=0.5),      # Random brightness/contrast adjustments
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),  # Random shift, scale, and rotation
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),  # Random hue, saturation, and value adjustments
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),  # Gaussian blur with random kernel size
], p=1.0)  # Apply the entire pipeline with probability 1.0

# Function to augment images in a directory
def augment_images(source_dir, dest_dir, num_augmentations=SAMPLES_AUG_IMG):
    """
    Augments images in a source directory and saves them to a destination directory.

    Args:
        source_dir (str): Path to the directory containing the original images.
        dest_dir (str): Path to the directory where augmented images will be saved.
        num_augmentations (int): Number of augmentations to generate per image.
    """
    for filename in os.listdir(source_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(source_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            for i in range(num_augmentations):
                augmented_image = transform(image=image)['image']
                augmented_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.jpg"
                augmented_path = os.path.join(dest_dir, augmented_filename)
                cv2.imwrite(augmented_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

# Create the augmented dataset directory structure
if not os.path.exists(augmented_dataset_path):
    os.makedirs(augmented_dataset_path)

# Iterate through classes and augment images
for class_name in os.listdir(dataset_path):
    class_source_dir = os.path.join(dataset_path, class_name)
    class_dest_dir = os.path.join(augmented_dataset_path, class_name)
    if not os.path.exists(class_dest_dir):
        os.makedirs(class_dest_dir)
    augment_images(class_source_dir, class_dest_dir)

print("Data augmentation complete!")

In [ ]:
# Directory with the images
test_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/test"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    seed=42,
    label_mode='int',
    validation_split=None,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

In [ ]:
# Directory with the images
train_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/augmented_train"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    seed = 42,
    label_mode='int',
    validation_split=None,
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
# Directory with the images
val_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/val"
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    seed = 42,
    label_mode='int',
    validation_split=None,
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
print(f"""
    ----------------------------------
    Dataset split:
        - train split: {len(train_ds)}
        - val split: {len(val_ds)}
        - test split: {len(test_ds)}
    ----------------------------------
    """
)

In [ ]:
#Caching the dataset

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import os
import seaborn as sns
main_folder = 'augmented_train'

In [ ]:
era_counts = {}

# Iterate through each subfolder (era)
for era_folder in os.listdir(main_folder):
    era_path = os.path.join(main_folder, era_folder)
    if os.path.isdir(era_path):
        # Count the number of images in the era folder
        era_counts[era_folder] = len([filename for filename in os.listdir(era_path) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Create a pandas DataFrame from the era_counts dictionary
import pandas as pd
df = pd.DataFrame({'Era': era_counts.keys(), 'Image Count': era_counts.values()})

# Create the bar plot using seaborn
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
ax = sns.barplot(x='Era', y='Image Count', data=df, palette='flare')  # Choose a color palette you like
for bar in range(len(ax.containers)):
    ax.bar_label(ax.containers[bar])
# Customize the plot
plt.title('Data Imbalance in Taylor Swift Eras Tour Images', fontsize=16)
plt.xlabel('Eras', fontsize=14)
plt.ylabel('Number of Images', fontsize=14)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()  # Adjust layout for better spacing

# Display the plot
plt.show()

In [ ]:
def get_weight_inverse_num_samples(num_of_classes, samples_per_classes, coeff=1):
    """
    Get the inverse number of samples for each class.
    Args:
        num_of_classes (int): Number of classes in the dataset.
        samples_per_classes (list): List of number of samples per class.
        power (int, optional): Power of the inverse number of samples. Defaults to 1.
    Returns:
        list: List of inverse number of samples for each class.
    """

    inverse_num_samples = []
    total_samples = sum(samples_per_classes)

    for i in range(num_of_classes):
        inverse_num_samples.append((total_samples / (num_of_classes * samples_per_classes[i])) * coeff)

    return inverse_num_samples

num_of_classes = 10
samples_per_classes = [era_counts['1989'], era_counts['Acoustic'], era_counts['Fearless'], era_counts['Folkmore'], era_counts['Lover'], era_counts['Midnights'], era_counts['Red'], era_counts['Reputation'], era_counts['Speak Now'], era_counts['TTPD']] 

weighted_classes = get_weight_inverse_num_samples(num_of_classes, samples_per_classes)


In [ ]:
class_indexes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
weights = dict(zip(class_indexes, weighted_classes))

In [ ]:
weights

In [ ]:
conv_base = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
)

In [ ]:
for layer in conv_base.layers[:-8]:
    layer.trainable = False

In [ ]:
conv_base.summary()

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

x = tf.keras.applications.efficientnet_v2.preprocess_input(inputs)

# The output of the base model is taken as the input for the new model
x = conv_base(x)
# This layer averages the spatial dimensions (height and width) of the output from the previous layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# Add a Dense layer with 512 units and 'relu' activation function to the model
x = tf.keras.layers.Dense(512, activation="relu")(x)
# This layer randomly sets a fraction of input units to 0 at each update during training time, which helps prevent overfitting
x = tf.keras.layers.Dropout(0.5)(x)
# This layer will output the probabilities for the 10 classes
outputs = tf.keras.layers.Dense(CLASSES, activation="softmax", kernel_regularizer=l2(0.001))(x)

#outputs = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
filepath="saved_models/all_VGG16_models/best_model_weighted_RESNET50_getty_aug.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
lr_plateau = callbacks.ReduceLROnPlateau(factor=0.8, monitor="val_accuracy", patience=5, verbose=1)
es = EarlyStopping(monitor='val_accuracy', patience=ES_EPOCHS)
callbacks_list = [checkpoint, es, lr_plateau]

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    class_weight=weights
)